<a href="https://colab.research.google.com/github/sobti/TSAI/blob/master/VAE/VAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import argparse
import os
import numpy as np
import math

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch
from google.colab import drive
import os
import sys
drive.mount('/content/gdrive/')
os.makedirs("/content/reconstructed", exist_ok=True)
from torchvision.utils import save_image

Mounted at /content/gdrive/


In [2]:
os.makedirs("/content/actual", exist_ok=True)
class VAE(nn.Module):
    def __init__(self, image_channels=3, h_dim=4096, z_dim=256):
        super(VAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(image_channels, 32, kernel_size=4, stride=2),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(),
            nn.Conv2d(64, 128, kernel_size=4, stride=2),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(),
            nn.Conv2d(128, 256, kernel_size=4, stride=2),
            #nn.LeakyReLU(),
            #nn.Conv2d(256, 512, kernel_size=4, stride=2),
            #nn.ReLU(),
            Flatten()
        )
        
        self.fc1 = nn.Linear(h_dim, z_dim)
        self.fc2 = nn.Linear(h_dim, z_dim)
        self.fc3 = nn.Linear(z_dim, h_dim)
        
        self.decoder = nn.Sequential(
            UnFlatten(),
            nn.ConvTranspose2d(h_dim, 128, kernel_size=4, stride=2),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(),
            nn.ConvTranspose2d(64, 32, kernel_size=5, stride=2),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(),
            nn.ConvTranspose2d(32, 16, kernel_size=5, stride=2),
            nn.BatchNorm2d(16),
            nn.LeakyReLU(),
            nn.ConvTranspose2d(16, image_channels, kernel_size=4, stride=2),
            nn.Sigmoid(),
        )
        
    def reparameterize(self, mu, logvar):
        #std = logvar.mul(0.5).exp_()
        # return torch.normal(mu, std)
        #esp = (torch.randn(*mu.size())).to('cuda')
        #z = mu + std * esp
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return eps * std + mu
        #return z
    def bottleneck(self, h):
        mu, logvar = self.fc1(h), self.fc2(h)
        z = self.reparameterize(mu, logvar)
        return z, mu, logvar

    def encode(self, x):
        h = self.encoder(x)
        z, mu, logvar = self.bottleneck(h)
        return z, mu, logvar

    def decode(self, z):
        z = self.fc3(z)
        z = self.decoder(z)
        return z

    def forward(self, x):
        z, mu, logvar = self.encode(x)
        z = self.decode(z)
        return z, mu, logvar


class Flatten(nn.Module):
    def forward(self, input):
        
        return input.view(input.size(0), -1)


class UnFlatten(nn.Module):
    def forward(self, input, size=4096):

        return input.view(input.size(0), size, 1, 1)

In [3]:
model = VAE(image_channels=3).to('cuda')

In [4]:
sys.path.append('/content/gdrive/My Drive/Colab Notebooks/')
os.chdir('/content/gdrive/My Drive/Colab Notebooks')
!7z x '/content/gdrive/My Drive/Colab Notebooks/red_cars.7z' -o'/content/data/'


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.00GHz (50653),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/gdrive/My Drive/Colab Notebooks/                                                   1 file, 3581431 bytes (3498 KiB)

Extracting archive: /content/gdrive/My Drive/Colab Notebooks/red_cars.7z
--
Path = /content/gdrive/My Drive/Colab Notebooks/red_cars.7z
Type = 7z
Physical Size = 3581431
Headers Size = 13842
Method = LZMA2:6m
Solid = +
Blocks = 1

  0%     87% 1       87% 939 - red_cars/car_0938.jpg                                 91% 1362 - red_cars/car_1361.jpg                                 

In [5]:
input=[]
label=[]
import numpy as np
from PIL import Image

count=1  # to restrict the data to few thousand


# Getting the BG list and replicate each background to 4000 


for root_cars, dirs_cars, files_cars in os.walk("/content/data/red_cars"):
   for cars in files_cars:
     check_for_Jpg=cars.split('.')
     if check_for_Jpg[-1].lower()=='jpg' or check_for_Jpg[-1].lower() =='jpeg':
      
        p= Image.open(root_cars +'/'+ cars)
        if p.mode == 'RGB':
         
         input.append(root_cars +'/'+ cars)

In [6]:
transform=transforms.Compose(
            [transforms.Resize((100,100)), transforms.ToTensor()])

In [7]:
from PIL import Image
class Myclass(torch.utils.data.Dataset):
    def __init__ (self,dataset,transform=None):
       #super(depthmask, self).__init__()
       self.dataset = dataset
       self.transform = transform
       

    def __getitem__ (self, idx): 
         out_dataset = Image.open(self.dataset[idx])
         if self.transform:
           out_dataset = self.transform(out_dataset)
           sample = {'images' : out_dataset }
         return sample
    
    def __len__ (self):
       return len(self.dataset)

In [8]:
train_dataset=Myclass(input,transform=transform)
for i in range(len(train_dataset)):
    sample = train_dataset[i]
    print(i,'-images-train:', sample['images'].size())

    if i == 3:
        break

0 -images-train: torch.Size([3, 100, 100])
1 -images-train: torch.Size([3, 100, 100])
2 -images-train: torch.Size([3, 100, 100])
3 -images-train: torch.Size([3, 100, 100])


In [9]:
SEED = 1
torch.cuda.empty_cache()
cuda = torch.cuda.is_available()
print("CUDA Available?", cuda)
torch.manual_seed(SEED)
if cuda:
  torch.cuda.manual_seed(SEED) 
dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=64,
                                          shuffle=True, num_workers=2)

CUDA Available? True


In [10]:
from torch.optim.lr_scheduler import StepLR
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001,betas=(0.9,0.999))
#optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9,0.999))
! pip install pytorch-msssim
from pytorch_msssim import ssim, ms_ssim, SSIM, MS_SSIM
#scheduler = StepLR(optimizer, step_size=50, gamma=0.1)

In [11]:
def loss_fn(recon_x, x, mu, logvar):
     MSE = F.mse_loss(recon_x.to('cuda'), x.to('cuda'))
     #MSE=F.binary_cross_entropy(recon_x.to('cuda'), x.to('cuda'),reduction='sum')
     #MSE =1 - ssim(recon_x.to('cuda'),x.to('cuda'),data_range=1.0, size_average=True)
    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
     KLD = torch.mean(-0.5 * torch.sum(1 + logvar - mu ** 2 - logvar.exp(), dim = 1), dim = 0)
     #KLD = -0.5 * torch.sum(1 + logvar - mu**2 -  logvar.exp())
     #KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
     return (MSE + (1/100000)*KLD),MSE,KLD

In [12]:
epochs=1500

for  epoch in range(epochs):
   for idx, (images) in enumerate(dataloader):
        #images = flatten(images)
        recon_images, mu, logvar = model(images['images'].to('cuda'))
        loss,BCE,KLD = loss_fn(recon_images, images['images'], mu, logvar)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        print("[Epoch %d/%d] [Batch %d/%d] [loss: %f]"
            % (epoch, epochs, idx, len(dataloader),loss.item()))
        #print(BCE,KLD,loss)
    
        batches_done = epoch * len(dataloader) + idx
        if batches_done % 5 == 0 and epoch % 30 == 0 :
            save_image(recon_images[:10], "/content/reconstructed/%d.png" % batches_done, nrow=5, normalize=True)
            save_image(images['images'][:10], "/content/actual/%d.png" % batches_done, nrow=5, normalize=True)
   #scheduler.step()

Streaming output truncated to the last 5000 lines.
[Epoch 1282/1500] [Batch 15/23] [loss: 0.010020]
[Epoch 1282/1500] [Batch 16/23] [loss: 0.009916]
[Epoch 1282/1500] [Batch 17/23] [loss: 0.009707]
[Epoch 1282/1500] [Batch 18/23] [loss: 0.010086]
[Epoch 1282/1500] [Batch 19/23] [loss: 0.010049]
[Epoch 1282/1500] [Batch 20/23] [loss: 0.009869]
[Epoch 1282/1500] [Batch 21/23] [loss: 0.009908]
[Epoch 1282/1500] [Batch 22/23] [loss: 0.009640]
[Epoch 1283/1500] [Batch 0/23] [loss: 0.010667]
[Epoch 1283/1500] [Batch 1/23] [loss: 0.009869]
[Epoch 1283/1500] [Batch 2/23] [loss: 0.009596]
[Epoch 1283/1500] [Batch 3/23] [loss: 0.009855]
[Epoch 1283/1500] [Batch 4/23] [loss: 0.009836]
[Epoch 1283/1500] [Batch 5/23] [loss: 0.009759]
[Epoch 1283/1500] [Batch 6/23] [loss: 0.010040]
[Epoch 1283/1500] [Batch 7/23] [loss: 0.009677]
[Epoch 1283/1500] [Batch 8/23] [loss: 0.010078]
[Epoch 1283/1500] [Batch 9/23] [loss: 0.009700]
[Epoch 1283/1500] [Batch 10/23] [loss: 0.009568]
[Epoch 1283/1500] [Batch 11/

In [18]:
import shutil
shutil.copytree('/content/reconstructed/','/content/gdrive/My Drive/Colab Notebooks/vae')

'/content/gdrive/My Drive/Colab Notebooks/vae'

In [ ]:
save_image(img[0][:10], "/content/reconstructed/ikmg.png", nrow=5, normalize=True)